In [3]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
import lightgbm as lgb
from sklearn.base import BaseEstimator, RegressorMixin
import os

class LGBMRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, **params):
        self.params = params
        self.model = None
    
    def fit(self, X, y):
        dtrain = lgb.Dataset(X, label=y)
        dvalid = lgb.Dataset(X, label=y, reference=dtrain)
        self.model = lgb.train(
            self.params, 
            dtrain, 
            num_boost_round=2000, 
            valid_sets=[dtrain, dvalid], 
            valid_names=['train', 'valid'],
            callbacks=[lgb.early_stopping(stopping_rounds=20)]
        )
        return self
    
    def predict(self, X):
        return self.model.predict(X)
    
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params.update(params)
        return self

# データ読み込み
normal_data_file = 'normal2.csv'
abnormal_data_file = 'anormal2.csv'

normal_data = pd.read_csv(normal_data_file)
abnormal_data = pd.read_csv(abnormal_data_file)

normal_data['label'] = 0
abnormal_data['label'] = 1
data = pd.concat([normal_data, abnormal_data], ignore_index=True)

# 一時的or永続的に補完説明変数から除外
temp_excluded_features = [] #'Model', 'cylinder', 'distance'
permanent_excluded_features = []+['outer_wear_Oil_Upper', 'S1_TOP', 'S1_Oil_Lower', 'Mating_angle_Oil_Upper', 'tension_TOP', 'updwn_surface_wear_TOP_Upper', 'updwn_surface_wear_TOP_Lower', 'outer_wear_2nd', 'Mating_angle_Oil_Lower', 'S1_2nd', 'S1_Oil_Upper', 'tension_Oil', 'distance', 'S1_insert_Oil_Upper', 'updwn_surface_wear_2nd_Upper', 'Mating_angle_TOP', 'outer_wear_Oil_Lower', 'S2_insert_Oil_Lower']

data_for_imputation = data.drop(columns=temp_excluded_features + permanent_excluded_features).copy()

# 基本情報
num_features = data_for_imputation.shape[1]
num_normal = normal_data.shape[0]
num_abnormal = abnormal_data.shape[0]
total_samples = num_normal + num_abnormal
target_num_feature = int(total_samples*0.1)

feature_fraction_ratio = target_num_feature / num_features
ffr = feature_fraction_ratio if feature_fraction_ratio < 1 else 1.0
print(f"全サンプル数: {total_samples}")
print(f"特徴量数: {num_features}")
print(f"目標特徴量数: {target_num_feature}")
print(f"特徴量数/全サンプル: {num_features/total_samples*100:.2f}%")

# LGBMのパラメータ設定
lgb_params = {'objective': 'regression', 'verbosity': -1, 'metric': 'rmse', 'feature_fraction': ffr, 'reg_alpha': 1.0, 'seed':42}
lgbm_regressor = LGBMRegressor(**lgb_params)

# IterativeImputerの設定
imputer = IterativeImputer(
    estimator=lgbm_regressor,
    max_iter=20,
    tol=1e-4,
    imputation_order='random',
    initial_strategy='mean')

# 欠損値がある列を特定
missing_cols = [col for col in data_for_imputation.columns if data_for_imputation[col].isnull().sum() > 0]

# 欠損変数の欠損行を補完
data_imputed_values = imputer.fit_transform(data_for_imputation[missing_cols])
data_for_imputation.loc[:, missing_cols] = data_imputed_values

# 補完後のデータにtemp_excluded_featuresを元の位置に戻す
for col in temp_excluded_features:
    data_for_imputation.insert(data.columns.get_loc(col), col, data[col])

data_imputed = data_for_imputation.copy()

# normal_dataとabnormal_dataに分ける
normal_data_imputed = data_imputed[data_imputed['label'] == 0].drop(columns=['label'])
abnormal_data_imputed = data_imputed[data_imputed['label'] == 1].drop(columns=['label'])

# CSVとして保存
normal_data_imputed.to_csv(f'{os.path.splitext(normal_data_file)[0]}_impute.csv', index=False)
abnormal_data_imputed.to_csv(f'{os.path.splitext(abnormal_data_file)[0]}_impute.csv', index=False)


全サンプル数: 44
特徴量数: 7
目標特徴量数: 4
特徴量数/全サンプル: 15.91%
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	train's rmse: 4.54141	valid's rmse: 4.54141
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	train's rmse: 0.520389	valid's rmse: 0.520389
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	train's rmse: 91.6095	valid's rmse: 91.6095
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	train's rmse: 97.5209	valid's rmse: 97.5209
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	train's rmse: 0.785969	valid's rmse: 0.785969
